# Evaluation test - Code

In [1]:
# Import statements
from datetime import datetime 
import logging
import pytz
import h5py
import csv

In [2]:
# Logging config
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

In [3]:
hdf_file_path = '1541962108935000000_167_838.h5'

In [4]:
# Task 1
digit_number = int(hdf_file_path[:18+1]) // (10**9) # String (nanosec) to int (sec)
dt = datetime.fromtimestamp(digit_number)

dt_utc = pytz.utc.localize(dt)
print("UTC: ", dt_utc)
dt_cern = pytz.timezone('CET').localize(dt)
print("Cern local time: ", dt_cern)

UTC:  2018-11-11 19:48:28+00:00
Cern local time:  2018-11-11 19:48:28+01:00


In [10]:
# Task 2
import io
def visit(name, obj):   
    if isinstance(obj, h5py.Dataset):
        features = dataToList(name, obj)
        
        with open('data.csv', 'a', newline='') as csvfile:
            csvwr = csv.writer(csvfile, delimiter=',', quotechar='"')
            csvwr.writerow(features)
        
def dataToList(name, obj):
    output = []
    
    nameSep = name.split('/')
    output.append('/'.join(nameSep[:-1]))
    output.append(nameSep[-1])

    try:
        output.append(str(obj.dtype))
    except TypeError as e:
        # TypeBitfieldID is not supported right now, see https://github.com/h5py/h5py/issues/109
        logger.info(f'"{name}" filetype not supported: {e}')
        output.append('Not supported')
    
    output.append(str(obj.shape))
    output.append(str(obj.size))
    
    return output

with h5py.File(hdf_file_path, 'r') as file:
    with open('data.csv', 'w', newline='') as csvfile:
        print('groups-name,dataset-name,dtype,shape,size', file=csvfile)
    
    file.visititems(visit)